In [23]:
# imports
import os, re, json, logging
from typing import List, Dict
import fitz          
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv

#find table

In [24]:
# ---------- Page discovery utilities ----------
MIN_CODES = 5                                           # tweak if a report is sparse
CODE_RE   = re.compile(r'\b(?:BP|GOV|SBM|IRO|E[1-5]|S[1-4]|G[1-3])[-–‐]?\d{1,2}\b', re.I)

def code_cnt(txt: str) -> int:
    "Count ESRS-style code tokens in a block of text."
    return len(CODE_RE.findall(txt))

def find_esrs_pages(pdf_path: str) -> List[int]:
    """
    Return a **1-based** list of pages that most likely hold the ESRS
    cross-reference table, using the heuristic you provided.
    """
    doc = fitz.open(pdf_path)

    # 1️⃣ “ESRS table of contents” segment
    toc_hits = [
        i for i, p in enumerate(doc)
        if "esrs table of contents" in p.get_text().lower()
        and code_cnt(p.get_text()) >= MIN_CODES
    ]
    if toc_hits:
        start  = toc_hits[-1]
        pages  = [start]
        for j in range(start + 1, len(doc)):
            if code_cnt(doc[j].get_text()) >= MIN_CODES:
                pages.append(j)
            else:
                break
        return [p + 1 for p in pages]                     # 1-based return

    # 2️⃣ densest contiguous block anywhere in the PDF
    qualifies = [code_cnt(p.get_text()) >= MIN_CODES for p in doc]
    blocks, i = [], 0
    while i < len(doc):
        if not qualifies[i]:
            i += 1; continue
        start = i
        while i < len(doc) and qualifies[i]:
            i += 1
        blocks.append(range(start, i))
    if not blocks:
        return []

    best = max(blocks, key=lambda r: len(r)*sum(code_cnt(doc[p].get_text()) for p in r))
    return [p + 1 for p in best]                          # 1-based list


In [25]:
import fitz

try:
    # Update path to local Reports directory
    doc = fitz.open("Reports/PhilipsFullAnnualReport2024-English.pdf")
    text = doc[242].get_text()  # zero-indexed
    print(text[:1000])  # show first 1000 characters
finally:
    # Ensure document is closed
    if 'doc' in locals():
        doc.close()

8.7 
ESRS cross-reference table
ESRS 2 
General disclosures
Disclosure Requirement BP-1 – General basis for preparation of sustainability
Sustainability statement: General basis for preparation
169 Limited Assurance
Disclosure Requirement BP-2 – Disclosures in relation to specific circumstances
Sustainability statement: General basis for preparation
169 Limited Assurance
Disclosure Requirement GOV-1 – The role of the administrative, management and 
supervisory bodies
SFDR/BRR
Environmental, Social and Governance: ESG governance
Supervisory Board report: composition, diversity and self-evaluation
Other Board-related matters: Diversity
40
64
268
Limited Assurance
Disclosure Requirement GOV-2 – Information provided to and sustainability matters 
addressed by the undertaking’s administrative, management and supervisory bodies
Environmental, Social and Governance: ESG governance
40 Limited Assurance
Disclosure Requirement GOV-3 - Integration of sustainability-related performance in incentiv

In [26]:
pdf_path = "Reports/PhilipsFullAnnualReport2024-English.pdf"
pages = find_esrs_pages(pdf_path)
print("ESRS pages:", pages)

ESRS pages: [243, 244, 245, 246, 247]


#LLM APY

In [27]:
# ---------- LLM extraction ----------
load_dotenv()  # Load environment variables

api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)

def extract_page_items(text: str, page_num: int, client: OpenAI) -> List[Dict]:
    """
    Call the model once for a single page of text and return list[dict].
    """
    prompt = f"""You are an ESRS disclosure analyzer. Extract **all** ESRS disclosures
from the text below (page {page_num}).  Return JSON like:
{{"items":[{{"code":"","title":"","page_reference":""}}]}}.

Text:
{text}"""
    try:
        resp  = client.chat.completions.create(
            model="gpt-4-turbo-preview",
            messages=[{"role":"user","content":prompt}],
            response_format={"type":"json_object"},
        )
        data  = json.loads(resp.choices[0].message.content)
        for itm in data.get("items", []):
            itm["source_page"] = page_num
        return data.get("items", [])
    except Exception as e:
        logging.error(f"LLM error on page {page_num}: {e}")
        return []


In [28]:


# ---------- LLM extraction ----------
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

# constants for the final table
COMPANY        = "Philips"   # or inject dynamically
CATEGORY       = "DR"
MAX_PAGE_COLS  = 5           # Page_ref1 … Page_ref5

def extract_page_items(text: str, page_num: int, client: OpenAI) -> pd.DataFrame:
    """
    Call GPT once for a single page and return a **ready-made DataFrame**
    with columns:  name, category, variable, value,
                   Page_ref1 … Page_ref{MAX_PAGE_COLS}
    """
    prompt = f"""You are an ESRS disclosure analyzer. Extract **all** ESRS disclosures
from the text below (page {page_num}).  Return JSON exactly like:
{{"items":[{{"code":"","title":"","page_reference":""}}]}}  — no extra keys.

Text:
{text}"""

    try:
        resp  = client.chat.completions.create(
            model="gpt-4o-mini",   # or "gpt-4o" / "gpt-4.1"
            messages=[{"role":"user","content":prompt}],
            response_format={"type":"json_object"},
        )
        data = json.loads(resp.choices[0].message.content)
        items: List[Dict] = data.get("items", [])

    except Exception as e:
        logging.error(f"LLM error on page {page_num}: {e}")
        items = []

    # ---------- reshape to “ideal” layout ----------
    if not items:         # nothing found on that page
        return pd.DataFrame(columns=["name","category","variable","value"]
                                      + [f"Page_ref{i}" for i in range(1, MAX_PAGE_COLS+1)])

    df = pd.DataFrame(items)

    # add constant / derived columns
    df = (df
          .assign(
              name     = COMPANY,
              category = CATEGORY,
              variable = df["code"],
              value    = 1,                     # 1 = disclosed
          )
          .drop(columns=["title"])              # keep if you still need it elsewhere
    )

    # split “76, 80” → Page_ref1 … Page_refN
    pages = (df["page_reference"]
             .str.split(",", expand=True)
             .apply(lambda col: col.str.strip())
             .rename(columns=lambda i: f"Page_ref{i+1}"))

    # ensure we always have Page_ref1 … Page_ref{MAX_PAGE_COLS}
    for i in range(pages.shape[1]+1, MAX_PAGE_COLS+1):
        pages[f"Page_ref{i}"] = ""

    df = pd.concat([df.drop(columns=["page_reference"]), pages], axis=1)

    # final column order
    df = df[["name","category","variable","value"]
            + [f"Page_ref{i}" for i in range(1, MAX_PAGE_COLS+1)]]

    return df.fillna("")        # blanks instead of NaN


In [31]:
pdf_path = "Reports/PhilipsFullAnnualReport2024-English.pdf"

pages = find_esrs_pages(pdf_path)
print("Chosen pages:", pages)

doc = fitz.open(pdf_path)

dfs = []                                            # << DataFrames live here
for p in pages:
    text = doc[p-1].get_text()          # PyMuPDF uses 0-based index
    df_page = extract_page_items(text, p, client)
    dfs.append(df_page)                 # << append, do NOT extend

full_table = pd.concat(dfs, ignore_index=True)
display(full_table.style.hide(axis="index"))
# full_table.to_excel("Philips_ESRS.xlsx", index=False)

Chosen pages: [243, 244, 245, 246, 247]


name,category,variable,value,Page_ref1,Page_ref2,Page_ref3,Page_ref4,Page_ref5
Philips,DR,BP-1,1,169,,,,
Philips,DR,BP-2,1,169,,,,
Philips,DR,GOV-1,1,40,,,,
Philips,DR,GOV-2,1,40,,,,
Philips,DR,GOV-3,1,76,,,,
Philips,DR,GOV-4,1,170,,,,
Philips,DR,GOV-5,1,40,,,,
Philips,DR,SBM-1,1,54,,,,
Philips,DR,SBM-2,1,54,,,,
Philips,DR,SBM-3,1,54,,,,
